In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random

In [2]:
from sklearn.metrics import r2_score
from sklearn.impute import KNNImputer

In [3]:
heart = pd.read_csv('HeartData.csv')
heart.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
float_vec = np.vectorize(float)
nan_feature = []
for icol in heart.columns:
    L = np.sum(heart.loc[:,icol]=='?')
    if (L==0):
        heart.loc[:,icol] = float_vec(heart.loc[:,icol].values)
    else:
        nan_feature.append(icol)
        heart.loc[heart[icol]!='?',icol] = float_vec(heart.loc[heart[icol]!='?',icol].values)

In [5]:
heart.to_csv('HeartData.csv')

In [6]:
feature_mean   = {}
feature_median = {}

for icol in heart.columns:
    
    if icol not in nan_feature:
        continue
    L = heart.loc[heart[icol]!='?',icol].values 
    feature_mean[icol] = np.mean(L)
    feature_median[icol] = np.median(L)

In [7]:
clean_index = np.array([])
dirty_index = np.array([])

for ind in list(heart.index):
    if (np.sum(heart.loc[ind,:]=='?')==0):
        clean_index = np.hstack([clean_index,int(ind)])
    else:
        dirty_index = np.hstack([dirty_index,int(ind)])

In [8]:
picked_cleanindex = random.sample(list(clean_index),int((1./3.)*clean_index.shape[0]))
picked_cleanindex = list(set(picked_cleanindex))

In [41]:
clean_heart_mean   = heart.loc[picked_cleanindex,:].copy()
clean_heart_median = heart.loc[picked_cleanindex,:].copy()
clean_heart_knn    = heart.copy()

In [10]:
clean_heart_mean.loc[:,nan_feature] = np.nan
clean_heart_median.loc[:,nan_feature] = np.nan
#clean_heart_knn.loc[:,nan_feature] = np.nan

In [11]:
for ifeature in nan_feature:
    clean_heart_mean.loc[:,ifeature]   = feature_mean[ifeature]
    clean_heart_median.loc[:,ifeature] = feature_median[ifeature]

In [12]:
def MSE(y_true,y_pred):
    return (np.sum((y_true-y_pred)**2))/(y_true.shape[0]**2)

In [13]:
r2_mean   = {}
r2_median = {}

for ifeature in nan_feature:
    
    true_val        = heart.loc[picked_cleanindex,ifeature].values
    temp_mean_val   = clean_heart_mean.loc[:,ifeature].values
    temp_median_val = clean_heart_median.loc[:,ifeature].values
    
    r2_mean[ifeature]   = MSE(true_val,temp_mean_val)
    r2_median[ifeature] = MSE(true_val,temp_median_val)

In [24]:
knn = KNNImputer()

In [25]:
knn.get_params()

{'add_indicator': False,
 'copy': True,
 'metric': 'nan_euclidean',
 'missing_values': nan,
 'n_neighbors': 5,
 'weights': 'uniform'}

In [26]:
clean_heart_knn.shape

(99, 14)

In [43]:
knn.get_params()

{'add_indicator': False,
 'copy': True,
 'metric': 'nan_euclidean',
 'missing_values': nan,
 'n_neighbors': 20,
 'weights': 'uniform'}

In [44]:
knn.set_params(n_neighbors = 5, missing_values='?')
knn_result = knn.fit_transform(clean_heart_knn)
clean_heart_knn.shape
#knn_result = pd.DataFrame(data = knn_result,columns=clean_heart_knn.columns, index = clean_heart_knn.index)

ValueError: could not convert string to float: '?'

In [37]:
X = [[1, 2, np.nan], [3, 4, 3], [np.nan, 6, 5], [8, 8, 7]]
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(X)

array([[1. , 2. , 4. ],
       [3. , 4. , 3. ],
       [5.5, 6. , 5. ],
       [8. , 8. , 7. ]])